In [1]:
from scipy.stats import wilcoxon
from scipy.stats import shapiro
import pandas as pd
import numpy as np
# import seaborn as sns
# import matplotlib.pyplot as plt
# import json

In [10]:
all= pd.read_csv('../AllWindowData.csv')
data=pd.DataFrame({'id':all["userId"],"case":all['caseNum'], 'Window Type':all['type'],'Environment': all[ "environment"],'Distance':all["distance"],'Window Anchor':all['windowMode'],'Transparency':all['transparency'],'Size':all['visual angle'],'Meeting Format':all['Meeting type'], 'Theta':all['theta'], 'Phi':all['phi'], 'Pitch':all['pitch'], 'Row':all['row'], 'Offset':all['offset'], 'x': all['x'], 'y': all['y'], 'z': all['z'], 'scale': all['scale']})
data['Window Type'] = data['Window Type'].replace({0: 'Shared Screen', 1: 'Speaker\'s Video'})
data['Window Anchor'] = data['Window Anchor'].replace({0: 'Path Anchor', 1: 'Head Anchor'})
data['Meeting Format'] = data['Meeting Format'].replace({1: 'Listening', 2: 'Discussion'})
data['Environment'] = data['Environment'].replace({'low': 'Low-Traffic', 'high': 'High-Traffic'})
data['Transparency']=1-data['Transparency']

In [11]:
def getStar(p_value):
    significant = ""
    if p_value < 0.001:
        significant = '***'
    elif p_value < 0.01:
        significant = '**'
    elif p_value < 0.05:
        significant = '*'
    return significant
def w(data, para, value):
    print(f'{"Factor":11s}\t{"Window Type":15s}\t\t{"Value":12s}\t{"P-value":7s}\t{"sig":3s}')
    grouped = data.groupby(['Window Type', para]).apply(lambda x: x[[para, 'Window Type']+value])
    # print(grouped)
    p = grouped[para].unique()
    # All
    a = grouped[(grouped[para]==p[0])]
    b = grouped[(grouped[para]==p[1])]
    for e in value:
        statistic, p_value = wilcoxon(np.array(a[e].tolist()), np.array(b[e].tolist()), alternative='two-sided')
        significant = getStar(p_value)
        print(f'{para:11s}\t{"All":15s}\t\t{e:12s}\t{p_value:.5f}\t{significant}')
    #
    for w in ['Shared Screen', 'Speaker\'s Video']:                
        a = grouped[(grouped['Window Type']==w) & (grouped[para]==p[0])]
        b = grouped[(grouped['Window Type']==w) & (grouped[para]==p[1])]
        for e in value:
            statistic, p_value = wilcoxon(np.array(a[e].tolist()), np.array(b[e].tolist()), alternative='two-sided')
            significant = getStar(p_value)
            # print(statistic, p[0])
            print(f'{para:11s}\t{w:15s}\t\t{e:12s}\t{p_value:.5f}\t{significant:3s}')
    print("") 

In [13]:
w(data, 'Environment', ['Size', 'Theta', 'Phi', 'Transparency'])
w(data, 'Meeting Format', ['Size', 'Theta', 'Phi', 'Transparency'])
w(data, 'Window Anchor', ['Size', 'Theta', 'Phi', 'Transparency'])

Factor     	Window Type    		Value       	P-value	sig
Environment	All            		Size        	0.03410	*
Environment	All            		Theta       	0.65783	
Environment	All            		Phi         	0.00017	***
Environment	All            		Transparency	0.00000	***
Environment	Shared Screen  		Size        	0.00513	** 
Environment	Shared Screen  		Theta       	0.21027	   
Environment	Shared Screen  		Phi         	0.00202	** 
Environment	Shared Screen  		Transparency	0.22065	   
Environment	Speaker's Video		Size        	0.98663	   
Environment	Speaker's Video		Theta       	0.50266	   
Environment	Speaker's Video		Phi         	0.02639	*  
Environment	Speaker's Video		Transparency	0.00000	***

Factor     	Window Type    		Value       	P-value	sig
Meeting Format	All            		Size        	0.51357	
Meeting Format	All            		Theta       	0.04880	*
Meeting Format	All            		Phi         	0.90084	
Meeting Format	All            		Transparency	0.61523	
Meeting Format	Shared Screen  	

In [5]:
def whp(data, para, value):
    print(f'{"Factor":11s}\t{"Window Anchor"}\t{"Window Type":15s}\t\t{"Value":12s}\t{"P-value":7s}\t{"sig":3s}')
    grouped = data.groupby(['Window Type', para]).apply(lambda x: x[[para, "Window Anchor", 'Window Type']+value])
    # print(grouped)
    p = grouped[para].unique()
    # All
    for anchor in ['Path Anchor', "Head Anchor"]: 
        a = grouped[(grouped[para]==p[0]) & (grouped['Window Anchor']==anchor)]
        b = grouped[(grouped[para]==p[1]) & (grouped['Window Anchor']==anchor)]
        for e in value:
            statistic, p_value = wilcoxon(np.array(a[e].tolist()), np.array(b[e].tolist()), alternative='two-sided')
            significant = getStar(p_value)
            print(f'{para:11s}\t{anchor:13s}\t{"All":15s}\t\t{e:12s}\t{p_value:.5f}\t{significant}')
    #
    for anchor in ['Path Anchor', "Head Anchor"]: 
        for w in ['Shared Screen', 'Speaker\'s Video']:                       
            a = grouped[(grouped['Window Type']==w) & (grouped['Window Anchor']==anchor) & (grouped[para]==p[0])]
            b = grouped[(grouped['Window Type']==w) & (grouped['Window Anchor']==anchor) & (grouped[para]==p[1])]
            for e in value:
                statistic, p_value = wilcoxon(np.array(a[e].tolist()), np.array(b[e].tolist()), alternative='two-sided')
                significant = getStar(p_value)
                print(f'{para:11s}\t{anchor:13s}\t{w:15s}\t\t{e:12s}\t{p_value:.4f}\t{significant:3s}')
    print("") 

In [6]:
whp(data, 'Environment', ['Size', 'Distance', 'Theta', 'Phi', 'Transparency'])
whp(data, 'Meeting Format', ['Size', 'Distance', 'Theta', 'Phi', 'Transparency'])

Factor     	Window Anchor	Window Type    		Value       	P-value	sig
Environment	Path Anchor  	All            		Size        	0.16553	
Environment	Path Anchor  	All            		Distance    	0.26377	
Environment	Path Anchor  	All            		Theta       	0.48808	
Environment	Path Anchor  	All            		Phi         	0.00160	**
Environment	Path Anchor  	All            		Transparency	0.00272	**
Environment	Head Anchor  	All            		Size        	0.08428	
Environment	Head Anchor  	All            		Distance    	0.35952	
Environment	Head Anchor  	All            		Theta       	0.14288	
Environment	Head Anchor  	All            		Phi         	0.04886	*
Environment	Head Anchor  	All            		Transparency	0.00019	***
Environment	Path Anchor  	Shared Screen  		Size        	0.0502	   
Environment	Path Anchor  	Shared Screen  		Distance    	0.5387	   
Environment	Path Anchor  	Shared Screen  		Theta       	0.7499	   
Environment	Path Anchor  	Shared Screen  		Phi         	0.0179	*  
Enviro

Environment	Head Anchor  	Speaker's Video		Size        	0.7548	   
Environment	Head Anchor  	Speaker's Video		Distance    	0.5642	   
Environment	Head Anchor  	Speaker's Video		Theta       	0.5805	   
Environment	Head Anchor  	Speaker's Video		Phi         	0.3015	   
Environment	Head Anchor  	Speaker's Video		Transparency	0.0003	***

Factor     	Window Anchor	Window Type    		Value       	P-value	sig
Meeting Format	Path Anchor  	All            		Size        	0.05181	
Meeting Format	Path Anchor  	All            		Distance    	0.96624	
Meeting Format	Path Anchor  	All            		Theta       	0.45245	
Meeting Format	Path Anchor  	All            		Phi         	0.94095	
Meeting Format	Path Anchor  	All            		Transparency	0.94937	
Meeting Format	Head Anchor  	All            		Size        	0.69813	
Meeting Format	Head Anchor  	All            		Distance    	0.37558	
Meeting Format	Head Anchor  	All            		Theta       	0.01454	*
Meeting Format	Head Anchor  	All            		Phi  

In [20]:
def wwt(data, value):
    print(f'{"Value":12s}\t{"P-value":7s}\t\t{"sig":3s}\t{"mean1":7s}\t{"mean2":7s}')
    grouped = data.groupby(['Window Type']).apply(lambda x: x[['Window Type']+value])

    # All
    a = grouped[(grouped['Window Type']=='Shared Screen')]
    b = grouped[(grouped['Window Type']=='Speaker\'s Video')]
    for e in value:
        statistic, p_value = wilcoxon(np.array(a[e].tolist()), np.array(b[e].tolist()), alternative='two-sided')
        significant = getStar(p_value)
        print(f'{e:12s}\t{p_value:.5f}\t\t{significant}\t{a[e].mean():.5f}\t{b[e].mean():.5f}')
    #
    # for w in ['Shared Screen', 'Speaker\'s Video']:                
    #     a = grouped[(grouped['Window Type']==w)]
    #     b = grouped[(grouped['Window Type']==w)]
    #     for e in value:
    #         statistic, p_value = wilcoxon(np.array(a[e].tolist()), np.array(b[e].tolist()), alternative='two-sided')
    #         significant = getStar(p_value)
    #         print(f'{w:15s}\t\t{e:12s}\t{p_value:.5f}\t{significant:3s}\t{a[e].mean():.5f}\t{b[e].mean():.5f}')
    print("") 

In [21]:
wwt(data, ['Size', 'Offset', 'Theta', 'Phi', 'Transparency'])

Value       	P-value		sig	mean1  	mean2  
Size        	0.00000		***	0.35421	0.19240
Offset      	0.00005		***	2.50064	2.37038
Theta       	0.00001		***	1.65888	1.47257
Phi         	0.07924			1.85226	1.81519
Transparency	0.49844			0.82223	0.81770

